# Transferencia de estilos a imágenes existente, utilizando redes neuronales

Decalaraciones iniciales de Keras e imagenes a utilizar

In [1]:
#Declaración de librerías (Keras para convertir imagen a array)
from keras.preprocessing.image import load_img, img_to_array

#Declaración de las rutas de la imagen base y la imagen que contiene el estilo (Directorio actual)
imgContent_path = 'big-ben-londres.jpg'
imgStyle_path = 'noche-estrellada.jpg'
#Obtención de las dimensiones que tiene la imagen base
width, height = load_img(imgContent_path).size
#Reescalamos la imagen a una altura de 200 pixeles y su correspondiente proporcional de ancho según los valores anteriores
#para evitar un procesamiento muy elevado
img_height = 200
img_width = int(width*img_height/height)

Using TensorFlow backend.


Declaración de librerías de Numpy y funciones para tratamiento de imágenes

In [11]:
#Importar librerías y funciones
import numpy as np
from keras.applications import vgg16

#Función para procesamiento de imagen (1)
def preprocess_image(image_path):
    #Se carga la imagen y se reescala con los valores calculados anteriormente
    img = load_img(image_path, target_size=(img_height, img_width))
    #La imagen es convertido en un array plano que contiene el valor de 3 canales de la imagen (RGB)
    img = img_to_array(img)
    #A la imagen anterior le es agregada una nueva dimensión o eje (0)
    img = np.expand_dims(img, axis=0)
    #Vgg16 es un modelo preentrenado de red neuronal perteneciente a ImageNet
    img = vgg16.preprocess_input(img)
    return img


def deprocess_image(x):
    #Ajustes de la imagen
    # Se remueve el el valor central (0) para cada pixel como dice la documentación de Keras 
    x[:, :, 0] += 103.939  
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    #Se realiza la conversión BGR -> RGB
    x = x[:, :, ::-1]  
    #Limita los valores de los canales de entrada entre 0 y 255 y en formato uint8 (Para solo tener valores válidos de
    #intensidad para cada pixel)
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [12]:
from keras import backend as K

# Se mide la diferencia entre las representaciones (mapas de rasgos) mas abstractas de dos imagenes
def Jcontent(features1, features2):
    return K.sum(K.square(features2 - features1))

# Metrica autosimilitud para los mapas de rasgos
def gram_matrix(x):
    #vectoriza respecto el ancho y la altura, no el canal.
    #Además, se incerta la capa de rasgos en el medio de la imagen original y se hace un "aplanamiento" para tener
    #un concentrado de las imágenes (autosimilitud)
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# Mide la diferencia entre auto-similitudes de los mapas de rasgos
def Jstyle(features1, features2):
    G1 = gram_matrix(features1)
    G2 = gram_matrix(features2)
    #channels = K.int_shape(style)[2]
    channels = 3
    size = img_height*img_width
    return K.sum(K.square(G2 - G1))/(4.*(channels**2)*(size**2))

# Metrica para medir la similitud entre pixels contiguos en horizotal y vertical
def Jtotalvariation(x):
    dh = K.square(x[:, :img_height-1, :img_width-1, :] - x[:, 1:, :img_width-1, :])
    dw = K.square(x[:, :img_height-1, :img_width-1, :] - x[:, :img_height-1, 1:, :])
    return K.sum(K.pow(dh + dw, 1.25))

In [14]:
#Se mandan a llamar las funciones anteriores (Preprocesamiento para mabas imágenes)
imgContent = K.constant(preprocess_image(imgContent_path))
imgStyle = K.constant(preprocess_image(imgStyle_path))
imgGen = K.placeholder((1, img_height, img_width, 3))
#Se concatenan tanto la imagen original, la del estilo y la que contendrá los rasgos en el eje cero
input_tensor = K.concatenate([imgContent,imgStyle,imgGen], axis=0)
#Se genera el modelo de red neuronal a partir de los mapas de rasgos y la imágenes usadas
#(Utilizando los pesos de la red preentrenada)
model = vgg16.VGG16(input_tensor=input_tensor,weights='imagenet',include_top=False)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [15]:
#Reescribimos los parámetros del modelo para darle mayor peso a los rasgos 
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
content_layer = 'block5_conv2'
style_layers = ['block1_conv1','block2_conv1','block3_conv1','block4_conv1','block5_conv1']
total_variation_weight = 1e-4
style_weight = 1e3
content_weight = 1e-2

In [16]:
# Construir la funcion de costo total (Para la predicción de estilo en cada pixel de la imagen)
loss = K.variable(0.)

layer_features = outputs_dict[content_layer]
imgContent_features = layer_features[0, :, :, :]
imgGen_features = layer_features[2, :, :, :]
loss += content_weight*Jcontent(imgContent_features,imgGen_features)

for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    imgStyle_features = layer_features[1, :, :, :]
    imgGen_features = layer_features[2, :, :, :]
    Jstyle_layer = Jstyle(imgStyle_features, imgGen_features)
    loss += (style_weight/len(style_layers))*Jstyle_layer

#La función de costo se determina a partir de las variaciones en la imagen 
loss += total_variation_weight*Jtotalvariation(imgGen)

In [17]:
grads = K.gradients(loss, imgGen)[0]
fetch_loss_and_grads = K.function([imgGen], [loss, grads])
class Evaluator(object):
    
# Funciones de seteo, cálculo de costo y gradientes 
    def __init__(self):
        self.loss_value = None
        self.grads_values = None
        
    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values
    
evaluator = Evaluator()

In [18]:
from scipy.optimize import fmin_l_bfgs_b
import matplotlib.pyplot as plt
import time

result_prefix = 'output'
iterations = 10
x = preprocess_image(imgContent_path)
x = x.flatten()

#Se realizan las iteraciones (10 en este caso) donde se obtiene el costo y los valores de todas las capas para cada pixel 
for i in range(iterations):
    print('Iniciando iteracion', i+1)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x, fprime=evaluator.grads, maxfun=20)
    
    print('Costo =', min_val)
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    end_time = time.time()
    print('Iteracion %d completada en %dseg' % (i+1, end_time - start_time))
    plt.imshow(img)
    plt.show()

Iniciando iteracion 1
Costo = 600583100000.0
Iteracion 1 completada en 167seg


<Figure size 640x480 with 1 Axes>

Iniciando iteracion 2
Costo = 186133820000.0
Iteracion 2 completada en 179seg


<Figure size 640x480 with 1 Axes>

Iniciando iteracion 3
Costo = 118775520000.0
Iteracion 3 completada en 182seg


...

<Figure size 640x480 with 1 Axes>

Iniciando iteracion 9
Costo = 48572457000.0
Iteracion 9 completada en 178seg


<Figure size 640x480 with 1 Axes>

Iniciando iteracion 10
Costo = 45329834000.0
Iteracion 10 completada en 171seg


<Figure size 640x480 with 1 Axes>

In [19]:
from imageio import imwrite
imwrite('Resultado.jpg', img)